In [68]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [69]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 7

5


In [70]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=50, n_estimators=15,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-10)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)'],
 ["['B_Weight_lbs', 'B_win_by_Decision_Unanimous', 'R_win_by_Decision_Split', 'R_draw', 'B_avg_TD_landed', 'sig_str_dif', 'B_current_win_streak', 'B_Height_cms', 'R_win_by_TKO_Doctor_Stoppage', 'R_win_by_Submission', 'win_streak_dif', 'R_longest_win_streak', 'R_losses', 'B_longest_win_streak', 'R_total_title_b

In [48]:
df = pd.read_csv("../data/ufc-master.csv")

In [49]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [50]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [51]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [52]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4080
4080
4080


In [53]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,label
200,Tim Means,Daniel Rodriguez,-345,285,28.985507,285.000000,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,SUB,Guillotine Choke,1
201,John Dodson,Nathaniel Wood,130,-150,130.000000,66.666667,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,KO/TKO,Punch,0
202,Jim Miller,Scott Holtzman,115,-135,115.000000,74.074074,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,1
203,Devin Clark,Dequan Townsend,-400,325,25.000000,325.000000,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,0
204,Casey Kenney,Merab Dvalishvili,150,-170,150.000000,58.823529,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4275,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,NaN,1
4276,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,Punch,0
4277,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,Punches,0
4278,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,label
0,Calvin Kattar,Dan Ige,-286,220,29.850746,245.000000,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,10.0,NaN,NaN,NaN,Red,U-DEC,NaN,0
1,Tim Elliott,Ryan Benoit,-141,112,80.000000,100.000000,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,0
2,Jimmie Rivera,Cody Stamann,-141,112,70.921986,112.000000,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,10.0,NaN,NaN,neither,U-DEC,NaN,0
3,Molly McCann,Taila Santos,-152,120,80.000000,100.000000,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,1
4,Abdul Razak Alhassan,Mounir Lazzez,-345,260,28.985507,260.000000,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Diego Sanchez,Michel Pereira,135,-155,135.000000,64.516129,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,DQ,NaN,0
196,Montana De La Rosa,Mara Romero Borella,-175,155,57.142857,155.000000,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,0
197,Brok Weaver,Kazula Vargas,-270,230,37.037037,230.000000,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,DQ,NaN,0
198,Rogerio Bontorin,Ray Borg,130,-150,130.000000,66.666667,2020-02-15,"Rio Rancho, New Mexico, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,1


In [54]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [55]:
models[1][model_num]

'GaussianNB(priors=None, var_smoothing=1e-09)'

In [56]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [57]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [58]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [59]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [60]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_6_27_20
GaussianNB()
['R_longest_win_streak', 'longest_win_streak_dif', 'reach_dif', 'avg_sub_att_dif', 'R_win_by_Decision_Unanimous', 'B_win_by_Submission', 'R_Weight_lbs', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'height_dif', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
0


In [61]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [62]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [19]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: 13132.930985319346
Feature: weight_class Score: 13171.819190811137
Feature: B_current_lose_streak Score: 14262.176445111585
Feature: B_avg_TD_landed Score: 17737.786454674104
Feature: R_avg_SIG_STR_pct Score: 19234.206685814326
The best feature was R_avg_SIG_STR_pct.  It scored 19234.206685814326
Current best score is: 19234.206685814326
Feature: R_Height_cms Score: 19613.670469803725
The best feature was R_Height_cms.  It scored 19613.670469803725
Current best score is: 19613.670469803725
NO IMPROVEMENT
FINAL BEST SCORE: 19613.670469803725

model_4
GaussianNB(var_smoothing=1e-12)
['R_Height_cms', 'R_avg_SIG_STR_pct', 'B_age', 'R_longest_win_streak', 'lose_streak_dif', 'ko_dif', 'R_win_by_Decision_Majority', 'longest_win_streak_dif', 'avg_sub_att_dif', 'R_win_by_Decision_Unanimous', 'R_Weight_lbs', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', '

(3137, 21)

(3137,)

(3137, 2)

(200, 21)

...

(200, 21)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-286,220,4.797224e-01,0.520278,0
1,-141,112,8.815230e-01,0.118477,0
2,-141,112,1.592695e-08,1.000000,0
3,-152,120,8.144075e-01,0.185592,1
4,-345,260,8.638328e-01,0.136167,1
...,...,...,...,...,...
195,135,-155,2.786509e-01,0.721349,0
196,-175,155,9.373866e-01,0.062613,0
197,-270,230,9.010857e-01,0.098914,0
198,130,-150,4.963952e-01,0.503605,1



          Number of matches: 200
          Number of bets: 195
          Number of winning bets: 126
          Number of losing bets: 69
          Number of underdog bets: 74
          Number of underdog wins: 35
          Number of underdog losses: 39
          Number of Favorite bets: 114
          Number of favorite wins: 87
          Number of favorite losses: 27
          Number of even bets: 7
          Number of even wins: 4
          Number of even losses: 3
          Profit: 2678.779290986818
          Profit per bet: 13.737329697368297
          Profit per match: 13.39389645493409
          
          
HI
HI
HI
HI
HI


In [63]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3248, 18)

(3248,)

(3248, 2)

(200, 18)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-286,220,5.141998e-01,0.485800,0
1,-141,112,7.994653e-01,0.200535,0
2,-141,112,1.060091e-08,1.000000,0
3,-152,120,7.666961e-01,0.233304,1
4,-345,260,8.213166e-01,0.178683,1
...,...,...,...,...,...
195,135,-155,2.662831e-01,0.733717,0
196,-175,155,9.233659e-01,0.076634,0
197,-270,230,8.276948e-01,0.172305,0
198,130,-150,5.338866e-01,0.466113,1



          Number of matches: 200
          Number of bets: 189
          Number of winning bets: 123
          Number of losing bets: 66
          Number of underdog bets: 64
          Number of underdog wins: 31
          Number of underdog losses: 33
          Number of Favorite bets: 119
          Number of favorite wins: 88
          Number of favorite losses: 31
          Number of even bets: 6
          Number of even wins: 4
          Number of even losses: 2
          Profit: 2567.61081275238
          Profit per bet: 13.585242395515237
          Profit per match: 12.838054063761899
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [64]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [65]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [66]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [67]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=50, n_estimators=15,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-10)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()'],
 ["['B_Weight_lbs', 'B_win_by_Decision_Unanimous', 'R_win_by_Decision_Split', 'R_draw', 'B_avg_TD_landed', 'sig_str_dif', 'B_current_win_streak', 'B_Height_cms', 'R_win_by_TKO_Doctor_Stoppage', 'R_win_by_Submission', 'win_streak_dif', 'R_longest_win_streak', 'R_losses', 'B_longest_win_streak', 'R_total_title_bouts', 'R_current_win_streak', 'title_bout', 'R_avg_SIG_STR_pct', 'longest_win_streak_